# Импортируем библиотеки и ключ openai из файла token

In [1]:
import os
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import warnings

token_file = open("../utils/token.txt", "r")
token = token_file.readline()
token_file.close()

os.environ["OPENAI_API_KEY"] = token

number_of_goods = 6

# Деление текста на фрагменты

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter


# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=100,
#     chunk_overlap=0,
#     separators = ['. ', "\n"]
# )
# splitted_data = splitter.split_text(coffee_zero)
# splitted_data
path = '../input/'
text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader(path, glob="plus1.txt", loader_cls=TextLoader,
                         loader_kwargs=text_loader_kwargs)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )"]
)
splitted_texts = splitter.split_documents(docs)
splitted_texts

[Document(page_content='отличная кофемашина, бюджетная для семьи 2 человека, простая в использовании.', metadata={'source': '../input/plus1.txt'}),
 Document(page_content='"1. Весьма удобно, после рожка - исчезли россыпи молотого кофе по всей кухне.', metadata={'source': '../input/plus1.txt'}),
 Document(page_content='\n2. Имеется нормальный капучинатор, но я не пользовался еще, судя по отзывам качество посредственное, но его хотя бы не надо так тщательно вымывать после каждого использования, баклашку с молоком отцепили и поставили в холодильник, пениться будет лучше.', metadata={'source': '../input/plus1.txt'}),
 Document(page_content='\n3. Несмотря на черный цвет - материалы не маркие и раз в неделю, можно просто стереть пыль даже сухой тряпкой.', metadata={'source': '../input/plus1.txt'}),
 Document(page_content='\n4. Вместимости под зерна кофе хватает на дня 3-4, а лоток с водой и жмыхом примерно раз в сутки чищу."', metadata={'source': '../input/plus1.txt'}),
 Document(page_conten

# Сохраняем БД, а потом пробуем прочитать :)

In [3]:
directory = ''
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=splitted_texts,
    embedding=embedding,
    persist_directory=directory
)

print(vectordb._collection.count()) # Проверка
vectordb.persist() # Пишем

# # Читаем
# vectordb = Chroma(
#     persist_directory=directory,
#     embedding_function=embedding
# )
# print(vectordb._collection.count()) # Проверка

/Users/kevin/anaconda3/envs/shop-agent/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


57


# Пишем ответ с помощью GPT

In [4]:
warnings.filterwarnings("ignore")

question = "Расскажи достоинства этой кофемашины"
docs_k = 65

from pprint import pprint as pp

retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": docs_k})#search_type = "mmr"

template = """Ты - полезный ИИ консультант для нашего магазина бытовой техники по продаже кофемашин.
Используйте следующие фрагменты контекста (Context), чтобы ответить на вопрос в конце (Question).
Если вы не знаете ответа, просто скажите, что не знаете, не пытайтесь придумывать ответ.
Сначала убедитесь, что прикрепленный текст имеет отношение к вопросу.
Если текст не имеет отношения к вопросу, просто скажите, что текст не имеет отношения.
Используйте максимум 15 предложений. Дайте ответ как можно более понятным, рассказывая кратко про все особенности именно данной кофемашины.
Context: {context}
Question: {question}""" 

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

#langchain.debug = True
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    max_tokens = 250)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    #chain_type="refine",
    #chain_type="map_reduce",
)
result = qa_chain({"query": question})
pp(result)
langchain.debug = False


Number of requested results 65 is greater than number of elements in index 57, updating n_results = 57


{'query': 'Расскажи достоинства этой кофемашины',
 'result': 'Данная кофемашина имеет следующие достоинства:\n'
           '\n'
           '1. Варит вкусный кофе с отличной пенкой.\n'
           '2. Легкая в использовании и освоении.\n'
           '3. Быстро готовит кофе.\n'
           '4. Легко моется и обслуживается.\n'
           '5. Надежная и качественная.\n'
           '6. Встроенная кофемолка для использования свежих зерен.\n'
           '7. Возможность настройки крепости и объема кофе.\n'
           '8. Автоматический капучинатор для приготовления капучино.\n'
           '9. Компактный размер, занимает немного места на кухне.\n'
           '10. Простой в использовании и обслуживании.\n'
           '11. Регулировка температуры и объема кофе.\n'
           '12. Долговечность и надежность.\n',
 'source_documents': [Document(page_content='Отличная кофемашина.\n"Варит вкусный кофе.', metadata={'source': '../input/plus1.txt'}),
                      Document(page_content='Ну однознач